In [1]:
from docx import Document
import torch
import transformers 
from haystack.document_stores import ElasticsearchDocumentStore
from haystack.nodes import BM25Retriever, TfidfRetriever, FilterRetriever, DensePassageRetriever
from haystack.nodes import FARMReader
from haystack.pipelines import ExtractiveQAPipeline
from haystack.utils import print_answers
import spacy
nlp = spacy.load("en_core_web_sm")
import re

In [2]:
import import_ipynb
import Test_models as test_models
from Load_and_Print import read_qas, print_result

importing Jupyter notebook from Test_models.ipynb
importing Jupyter notebook from Load_and_Print.ipynb


##### After testing differrent models for retrivers and readers, model 3 has been chosen.
##### Based on model 3, test whether to to tokenize the text of the contract.

In [3]:
def process_contract1(file_dir):
    contract = Document(file_dir)
    
    contract_main = ""
    
    for para in contract.paragraphs:
        if check_para(para.text):
            contract_main = contract_main + para.text
    
    return contract_main

In [4]:
def process_contract2(file_dir):
    contract = Document(file_dir)
    
    contract_main = ""
    
    for para in contract.paragraphs:
        if check_para(para.text):
            contract_main = contract_main + para.text
    
    return tokenize(contract_main)

In [5]:
def check_para(p):
    if p.lower().startswith('ex') and not p == '':
        return False
    elif len(p)<=5:
        return False
    elif 'WITNESS ' in p or 'in witness' in p or 'signature' in p or 'signatures' in p:
        return False
    else:
        return True

In [6]:
def tokenize(text):
    tokenized_texts = []
    text = re.sub("[^a-zA-Z0-9]", " ", text)
        
    tokens = nlp(text)
    for token in tokens:
        if not token.is_stop and not token.is_punct:
            tokenized_texts.append(token.lemma_.lower())
    return " ".join(tokenized_texts)
    

In [7]:
qas = read_qas("./Questions.docx")

In [8]:
def test1():
    qas = read_qas('Questions.docx')
    for i in range(0,3):
        input_dir = f"./Agreements/Test Agreement {i+1}.docx"
        contract1 = process_contract1(input_dir)
        print(f"Contract{i+1}")
        doc_store = test_models.set_doc_store(contract1, 1)
        ans = test_models.test_model3(qas, contract1, doc_store)
        doc_store.delete_index(str(1))

In [9]:
def test2():
    qas = read_qas('Questions.docx')
    for i in range(0,3):
        input_dir = f"./Agreements/Test Agreement {i+1}.docx"
        contract2 = process_contract2(input_dir)
        print(f"Contract{i+1}")
        doc_store = test_models.set_doc_store(contract2, 2)
        ans = test_models.test_model3(qas, contract2, doc_store)
        doc_store.delete_index(str(2))

In [10]:
if __name__ == '__main__':
    test1()

Contract1
qs1


Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 2/2 [00:19<00:00,  9.99s/ Batches]



Query: Highlight the parts (if any) of this contract related to "Document Name" that should be reviewed by a lawyer. Details: The name of the contract.
Answers:
[   {   'answer': 'Product Collaboration Agreement',
        'context': 'vember 8, 2019.“Supplier” has the meaning set forth in the '
                   'Preamble.“Term” has the meaning set forth in\xa0Section '
                   '8(a).“Territory” means Canada and the United States, but '
                   'excluding the following non-state territories and '
                   'possessions of the United States: Puerto Rico, American '
                   'Samoa, Guam, Northern Mariana Islands and U.S. Virgin '
                   'Islands.“Third Party” means any person other than '
                   'Supplier, Customer and their respective '
                   'Affiliates.2.\xa0\xa0\xa0\xa0Purchase and Supply of '
                   'Products.(a)\xa0\xa0\xa0\xa0Products. During the Term, and '
                   'subject to t

Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 2/2 [00:20<00:00, 10.45s/ Batches]
ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-31556, -30933) with a span answer. 



Query: Highlight the parts (if any) of this contract related to "Parties" that should be reviewed by a lawyer. Details: The two or more parties who signed the contract.
Answers:
[   {   'answer': 'SunPower Corporation, a corporation organized and existing '
                  'under the laws of the State of Delaware, USA (“Customer”), '
                  'and Maxeon Solar Technologies, Ltd.,',
        'context': 'MASTER SUPPLY AGREEMENTThis Master Supply Agreement (this '
                   '“Agreement”), dated as of February 14, 2022 (the '
                   '“Effective Date”), is by and between SunPower Corporation, '
                   'a corporation organized and existing under the laws of the '
                   'State of Delaware, USA (“Customer”), and Maxeon Solar '
                   'Technologies, Ltd., a corporation organized and existing '
                   'under the laws of Singapore ( “Supplier”). Customer and '
                   'Supplier may also be referred to indi

...


Query: Highlight the parts (if any) of this contract related to "Cap On Liability" that should be reviewed by a lawyer. Details: Does the contract include a cap on liability upon the breach of a party’s obligation? This includes time limitation for the counterparty to bring claims or maximum amount for recovery.
Answers:
[   {   'answer': 'rovisions of this Agreement will not survive the expiration '
                  'or earlier termination of this Agreement.\xa0\xa0\xa0\xa0'
                  '38\xa0\xa0\xa0\xa017.5\xa0\xa0\xa0\xa0Notices. All notices, '
                  'requests, consents, claims, demands, waivers and other '
                  'communications under this Agreement (each, a "Notice") must '
                  'be in writing and addressed to the other Party at its '
                  'address set forth b',
        'context': 'Manufacturing Supply AgreementThis Manufacturing Supply '
                   'Agreement, dated as of August 9, 2021 (the “Agreement”), '
      

Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 5/5 [01:02<00:00, 12.44s/ Batches]
ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-4140, -4010) with a span answer. 
ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-32168, -31909) with a span answer. 



Query: Highlight the parts (if any) of this contract related to "Warranty Duration" that should be reviewed by a lawyer. Details: What is the duration of any\xa0 warranty against defects or errors in technology, products, or services\xa0 provided under the contract?
Answers:
[   {   'answer': 'delay or failure of either Party to perform its obligations '
                  'under this Agreement will be excused to the extent that the '
                  'delay or f',
        'context': 'Manufacturing Supply AgreementThis Manufacturing Supply '
                   'Agreement, dated as of August 9, 2021 (the “Agreement”), '
                   'is entered into by and between Shenzhen Fastrain '
                   'Technology Co., Ltd., a corporation formed under the laws '
                   'of the P.R.C. having an address at Room 401, Plant No. 2, '
                   'Baolong 4th Rd., Longgang District, Shenzhen, China '
                   '(P.R.C.), 518116 (“Shenzhen Fastrain”), togethe

Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 5/5 [01:02<00:00, 12.52s/ Batches]
ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-30323, -30156) with a span answer. 
ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-27780, -27590) with a span answer. 
ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-32286, -32074) with a span answer. 



Query: Highlight the parts (if any) of this contract related to "Insurance" that should be reviewed by a lawyer. Details: Is there a requirement for insurance that must be maintained by one party for the benefit of the counterparty?
Answers:
[   {   'answer': 'ecomes generally available to and known by the public other '
                  'than as a result of, directly or indirectly, any breach of '
                  'this Section 13 by the Receiving Party or any of',
        'context': 'Manufacturing Supply AgreementThis Manufacturing Supply '
                   'Agreement, dated as of August 9, 2021 (the “Agreement”), '
                   'is entered into by and between Shenzhen Fastrain '
                   'Technology Co., Ltd., a corporation formed under the laws '
                   'of the P.R.C. having an address at Room 401, Plant No. 2, '
                   'Baolong 4th Rd., Longgang District, Shenzhen, China '
                   '(P.R.C.), 518116 (“Shenzhen Fastrain”), toge

In [11]:
if __name__ == '__main__':
    test2()

Contract1
qs1


Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 2/2 [00:14<00:00,  7.39s/ Batches]



Query: Highlight the parts (if any) of this contract related to "Document Name" that should be reviewed by a lawyer. Details: The name of the contract.
Answers:
[   {   'answer': 'master supply agreement',
        'context': 'master supply agreementthis master supply agreement     '
                   'agreement     date february 14   2022     effective '
                   'date     sunpower corporation   corporation organize exist '
                   'law state delaware   usa    customer     maxeon solar '
                   'technologies   ltd    corporation organize exist law '
                   'singapore     supplier     customer supplier refer '
                   'individually   party     parties   backgrounda customer   '
                   'directly indirectly   design   market   sell product use '
                   'solar industry territory provide service solar industry '
                   'customer territory b supplier   directly indirectly   '
                   'des

Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 2/2 [00:15<00:00,  7.97s/ Batches]



Query: Highlight the parts (if any) of this contract related to "Parties" that should be reviewed by a lawyer. Details: The two or more parties who signed the contract.
Answers:
[   {   'answer': 'sunpower corporation   corporation organize exist law state '
                  'delaware   usa    customer     maxeon solar technologies   '
                  'ltd    corporation organize exist law singapore     '
                  'supplier     customer supplier',
        'context': 'master supply agreementthis master supply agreement     '
                   'agreement     date february 14   2022     effective '
                   'date     sunpower corporation   corporation organize exist '
                   'law state delaware   usa    customer     maxeon solar '
                   'technologies   ltd    corporation organize exist law '
                   'singapore     supplier     customer supplier refer '
                   'individually   party     parties   backgrounda customer   

...


Query: Highlight the parts (if any) of this contract related to "Cap On Liability" that should be reviewed by a lawyer. Details: Does the contract include a cap on liability upon the breach of a party’s obligation? This includes time limitation for the counterparty to bring claims or maximum amount for recovery.
Answers:
[   {   'answer': 'good compare price product agreement    b   require '
                  'fastrain deliver emcore finish product   work process part '
                  'material produce acquire work agreement capability fastrain '
                  'price products hereunder follow negotiation premise '
                  'capacity     c   require fastrain provide product source '
                  'quantity time determine party price prod',
        'context': 'manufacture supply agreementthis manufacturing supply '
                   'agreement   date august 9   2021     agreement     enter '
                   'shenzhen fastrain technology co    ltd    corporation 

Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 3/3 [00:47<00:00, 15.70s/ Batches]
ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-5548, -5388) with a span answer. 
ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-11159, -11080) with a span answer. 



Query: Highlight the parts (if any) of this contract related to "Warranty Duration" that should be reviewed by a lawyer. Details: What is the duration of any\xa0 warranty against defects or errors in technology, products, or services\xa0 provided under the contract?
Answers:
[   {   'answer': 'ailable   apart say act 17 16     dispute resolution   '
                  'dispute   controversy claim arise relate agreement   '
                  'breach   termination invalidity hereof       dispute',
        'context': 'manufacture supply agreementthis manufacturing supply '
                   'agreement   date august 9   2021     agreement     enter '
                   'shenzhen fastrain technology co    ltd    corporation form '
                   'law p r c   have address room 401   plant   2   baolong '
                   '4th rd    longgang district   shenzhen   china   p r c     '
                   '518116    shenzhen fastrain     hong kong fastrain company '
                   'lim

Inferencing Samples: 100%|█████████████████████████████████████████████████████████| 3/3 [00:46<00:00, 15.66s/ Batches]
ERROR:haystack.modeling.model.predictions:Invalid end offset: 
(-17330, -17193) with a span answer. 



Query: Highlight the parts (if any) of this contract related to "Insurance" that should be reviewed by a lawyer. Details: Is there a requirement for insurance that must be maintained by one party for the benefit of the counterparty?
Answers:
[   {   'answer': 'ual property rights   mean industrial intellectual property '
                  'right comprise relate      patents    b   trademark',
        'context': 'hority   hytera hk   meaning set forth preamble '
                   'agreement   hong kong financial reporting standards   mean '
                   'hong kong financial reporting standards   hong kong '
                   'accounting standards   hkas   interpretation issue hong '
                   'kong institute certified public accountants      4      '
                   'incoterm   mean incoterm 2010   eighth set pre define '
                   'international contract term publish international chamber '
                   'commerce relate international commercial law  